In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta

In [5]:
def read_ids_logs():
    url = 'https://raw.githubusercontent.com/data-challengers/DC4/b0c7eb4c6e4aef805bb78a2f714534c8c8915a61/DC4-data/IDS-logs.csv'
    df = pd.read_csv(url, parse_dates=[0], infer_datetime_format=True)
    # drop random terrible empty columns, curse you bad csvs
    df.drop(df.columns[[11, 12, 13,14,15,16,17,18,19,20]], axis=1, inplace=True)
    df.columns = df.columns.str.replace(' ', '')
    return df

df = read_ids_logs()
print(df)

                     time      sourceIP  sourcePort          destIP  destPort  \
0     2012-04-05 17:55:00  172.23.1.101        1101     172.23.0.10       139   
1     2012-04-05 17:55:00  172.23.1.101        1101     172.23.0.10       139   
2     2012-04-05 17:55:00  172.23.1.101        1104     172.23.0.10       139   
3     2012-04-05 17:55:00  172.23.1.101        1104     172.23.0.10       139   
4     2012-04-05 17:56:00  172.23.0.212        1222     172.23.0.10       445   
...                   ...           ...         ...             ...       ...   
35942 2012-04-07 08:59:00    10.32.5.51        6667  172.23.134.216      3850   
35943 2012-04-07 08:59:00    10.32.5.55        6667  172.23.134.215      3852   
35944 2012-04-07 08:59:00    10.32.5.53        6667  172.23.134.219      3857   
35945 2012-04-07 08:59:00    10.32.5.56        6667   172.23.134.78      3867   
35946 2012-04-07 08:59:00    10.32.5.54        6667  172.23.128.118      2834   

                         cl

/Users/ester/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (11,13,15,17,18,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
def get_internal_sources(df):
    # try to isolate first IP num for visited sites
    df['pos'] = df['sourceIP'].str.find('.')
    # use position to slice sourceIP
    df['ip_source'] = df.apply(lambda x: x['sourceIP'][0:x['pos']],axis=1)
    print(df['ip_source'].unique())

    # Create boolean internal? var
    df['internal_source'] = np.where(df['ip_source']=='172', True, False)
    df.drop(['pos', 'ip_source'], axis=1, inplace=True)

get_internal_sources(df)
print(df)

['172' '10']
                     time      sourceIP  sourcePort          destIP  destPort  \
0     2012-04-05 17:55:00  172.23.1.101        1101     172.23.0.10       139   
1     2012-04-05 17:55:00  172.23.1.101        1101     172.23.0.10       139   
2     2012-04-05 17:55:00  172.23.1.101        1104     172.23.0.10       139   
3     2012-04-05 17:55:00  172.23.1.101        1104     172.23.0.10       139   
4     2012-04-05 17:56:00  172.23.0.212        1222     172.23.0.10       445   
...                   ...           ...         ...             ...       ...   
35942 2012-04-07 08:59:00    10.32.5.51        6667  172.23.134.216      3850   
35943 2012-04-07 08:59:00    10.32.5.55        6667  172.23.134.215      3852   
35944 2012-04-07 08:59:00    10.32.5.53        6667  172.23.134.219      3857   
35945 2012-04-07 08:59:00    10.32.5.56        6667   172.23.134.78      3867   
35946 2012-04-07 08:59:00    10.32.5.54        6667  172.23.128.118      2834   

              

In [7]:
classifications = df['classification'].unique()
print('Unique classifications: ', classifications)

# create classification specific dfs
general_protocol = df.loc[df['classification'] == ' Generic Protocol Command Decode']
privacy_violations = df.loc[df['classification'] == ' Potential Corporate Privacy Violation']
leaks = df.loc[df['classification'] == ' Attempted Information Leak']
poten_bad = df.loc[df['classification'] == ' Potentially Bad Traffic']
misc = df.loc[df['classification'] == ' Misc activity']
# Check if there are any NA classifications
na_class = df[df['classification'].isnull()]
print(na_class)

Unique classifications:  [' Generic Protocol Command Decode'
 ' Potential Corporate Privacy Violation' ' Misc activity'
 ' Attempted Information Leak' ' Potentially Bad Traffic']
Empty DataFrame
Columns: [time, sourceIP, sourcePort, destIP, destPort, classification, priority, label, packetinfo, packetinfocont'd, xref, internal_source]
Index: []


In [8]:
# get percentage of entries that are from internal sources
percent = df['internal_source'].value_counts(normalize=True) * 100
print('Full df: internal sources percentages: \n',percent)

# percent of leaks
percent = leaks['internal_source'].value_counts(normalize=True) * 100
print('\nLeaks df: internal sources percentages: \n',percent)

# percent of privacy_violations
percent = privacy_violations['internal_source'].value_counts(normalize=True) * 100
print('\nprivacy_violations df: internal sources percentages: \n',percent)

# percent of poten_bad
percent = poten_bad['internal_source'].value_counts(normalize=True) * 100
print('\nPotentially bad df: internal sources percentages: \n',percent)

# percent of misc
percent = misc['internal_source'].value_counts(normalize=True) * 100
print('\nmisc df: internal sources percentages: \n',percent)

# percent of poten_bad
percent = general_protocol['internal_source'].value_counts(normalize=True) * 100
print('\ngeneral_protocol df: internal sources percentages: \n',percent)


Full df: internal sources percentages: 
 True     70.361922
False    29.638078
Name: internal_source, dtype: float64

Leaks df: internal sources percentages: 
 True    100.0
Name: internal_source, dtype: float64

privacy_violations df: internal sources percentages: 
 True    100.0
Name: internal_source, dtype: float64

Potentially bad df: internal sources percentages: 
 True    100.0
Name: internal_source, dtype: float64

misc df: internal sources percentages: 
 False    99.943715
True      0.056285
Name: internal_source, dtype: float64

general_protocol df: internal sources percentages: 
 True    100.0
Name: internal_source, dtype: float64
